In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
%%capture
import os
import numpy as np
import pandas as pd
import scanpy as sc
import scipy as sp
import anndata
from tqdm.notebook import tqdm
from utilities import sampleQC

In [3]:
cwd = os.getcwd()
cwd

'/Users/charlesdai/dev/FigureOneLab/projects/microglia_charles'

In [4]:
sample_donor = pd.read_csv(os.path.join(cwd, "data/kracht/SraRunTable.txt"))[["Sample Name", "donor/fetus"]]
donor_meta = pd.read_excel(os.path.join(cwd, "data/kracht/aba5906_tables_s1_s10.xlsx"), sheet_name=0, header=1)
sample_meta = sample_donor.merge(donor_meta, left_on="donor/fetus", right_on="ID", how="inner")
sample_meta.head()

,Sample Name,donor/fetus,ID,Gestational age (weeks + days),Gestational week,Sex,ATAC,Single cell
0,GSM4214849,2018-32,2018-32,9+4,9,F,Y,Y
1,GSM4214850,2018-33,2018-33,10+6,10,F,Y,N
2,GSM4214851,2018-34,2018-34,16+0,16,F,Y,Y
3,GSM4214852,2018-36,2018-36,11+3,11,M,Y,Y
4,GSM4214853,2018-37,2018-37,15+0,15,F,Y,Y


In [5]:
def to_days(age):
    weeks, days = map(int, age.split("+"))
    return weeks * 7 + days
sample_meta = sample_meta.rename(columns={"Sample Name":"SampleID", "ID":"PatientID", "Gestational age (weeks + days)":"AgeDay"})
sample_meta = sample_meta[["SampleID", "PatientID", "AgeDay", "Sex"]]
sample_meta["AgeDay"] = sample_meta["AgeDay"].apply(to_days)
sample_meta.head()

,SampleID,PatientID,AgeDay,Sex
0,GSM4214849,2018-32,67,F
1,GSM4214850,2018-33,76,F
2,GSM4214851,2018-34,112,F
3,GSM4214852,2018-36,80,M
4,GSM4214853,2018-37,105,F


In [12]:
ensembl_map = sc.queries.biomart_annotations("hsapiens", ["ensembl_gene_id", "hgnc_symbol"])
ensembl_map = ensembl_map.set_index("ensembl_gene_id")["hgnc_symbol"].to_dict()

In [13]:
filenames = [fname for fname in os.listdir('data/kracht/GSE141862_RAW') if 'completeCounts' in fname]

In [15]:
def process_sample(fname, log=False):
    sample_name = fname[:10]
    counts = pd.read_csv(
        os.path.join(cwd, os.path.join("data/kracht/GSE141862_RAW", fname)), 
        compression="gzip", sep="\t", header=0
    ).set_index("gene")

    # Set names
    counts.index.name = "Gene"
    counts.columns.name = "Barcode"

    # Map gene names to HGNC, delete duplicates
    counts.index = counts.index.map(ensembl_map)
    counts = counts[counts.index.notna()]
    counts = counts[~counts.index.duplicated()]
    counts = counts.transpose()

    # Create anndata object and set to sparse matrix
    adata = anndata.AnnData(counts)
    adata.X = sp.sparse.csr_matrix(adata.X)

    # Create observation matrix
    meta = sample_meta[sample_meta["SampleID"] == sample_name]
    adata.obs = pd.DataFrame(data=np.repeat(meta.values, adata.obs.shape[0], axis=0), index=adata.obs.index, columns=meta.columns)
    return adata

In [31]:
adatas = []
for fname in tqdm(filenames):
    adatas.append(process_sample(fname))
        
print(adatas[0])
display(adatas[0].obs)

  0%|          | 0/239 [00:00<?, ?it/s]

AnnData object with n_obs × n_vars = 81 × 14038
    obs: 'SampleID', 'PatientID', 'AgeDay', 'Sex'


,SampleID,PatientID,AgeDay,Sex
Barcode,,,,
AAATACACTC,GSM4214916,2018-42,105,F
AACCATCTGG,GSM4214916,2018-42,105,F
AAGACGTGAG,GSM4214916,2018-42,105,F
AAGCATCGAG,GSM4214916,2018-42,105,F
AATAATATCG,GSM4214916,2018-42,105,F
...,...,...,...,...
TGTTGAGCTA,GSM4214916,2018-42,105,F
TTAAGACGGA,GSM4214916,2018-42,105,F
TTAATCACTA,GSM4214916,2018-42,105,F


In [32]:
adata = anndata.concat(adatas, join="outer", fill_value=0)
adata.obs["AgeDay"] = adata.obs["AgeDay"].astype(int)
adata.obs_names_make_unique()

/Users/charlesdai/dev/FigureOneLab/.venv/lib/python3.10/site-packages/anndata/_core/anndata.py:1906: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [35]:
adata = sampleQC(adata, percent_cells_threshold=0.1, percent_mt_threshold=0)
adata

Before QC: 16681 cells, 20325 genes
After QC: 16669 cells, 20323 genes


AnnData object with n_obs × n_vars = 16669 × 20323
    obs: 'SampleID', 'PatientID', 'AgeDay', 'Sex', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'percent_mt'
    var: 'n_cells', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'

In [43]:

data.write(os.path.join(cwd, "out/kracht_adata.h5ad"))

In [36]:
adata.obs = adata.obs.astype(str)
adata.obs.dtypes

SampleID     object
PatientID    object
AgeDay       object
Sex          object
n_genes      object
dtype: object